In [ ]:
import pandas as pd
import numpy as np
from pandas import melt

import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.offline as ply
ply.init_notebook_mode(connected=True)

#colors for Sankey diagram
#https://plotly.com/python/discrete-color/
import plotly.express as px

https://office.wikimedia.org/wiki/Quarters

In [ ]:
all_funded = pd.read_csv('inputs/data-all-funded.csv')
year_roll_up_df = pd.read_csv('inputs/year_roll_up.csv')

In [ ]:
# clean all_funded

#drop rows without USD amount
df = all_funded[all_funded['USD over grant life'] != 0]
df = df[df['USD over grant life'].notnull()]

#include only Fiscal years ending 2008-2022
df = df[(df['Fiscal year ending'] >= 2008) & (df['Fiscal year ending'] <= 2022)]

#fix datatype for USD over grant life column
df['USD over grant life'] = df['USD over grant life'].str.replace(',', '')
df['USD over grant life'] = df['USD over grant life'].str.replace('$', '')
df['USD over grant life'] = df['USD over grant life'].astype('float')

#columns to datetime
df['Approved on'] =  pd.to_datetime(df['Approved on'], errors = 'coerce')
df['Executed on'] =  pd.to_datetime(df['Executed on'], errors = 'coerce')

#create a column of count_prior_grants for each grantee
df['grant_count'] = df.groupby('Grantee').cumcount() + 1
df["total_grantee_grants"] = df.groupby('Grantee') ['Approved on'].transform('count')+1 #to start the count at 

df['counter'] = range(len(df))

#assign unique IDs
df['id'] = df.groupby('Grantee').ngroup()

In [ ]:
df['Fiscal year ending'].unique()

In [ ]:
df['Program'].unique()
grants_df = df[df['grant_count'] != 'Conference']

In [ ]:
grants_df[grants_df['Fiscal year ending']==2020]['Program'].unique()

In [ ]:
grants_df = df[df['Program'] != 'Conference']

## What is the geo-distribution of new grantees over the years? 

In [ ]:
t = pd.pivot_table(data=df[df['grant_count'] == 1][['Fiscal year ending', 'Grantee', 'Country']], 
                   index='Country', columns='Fiscal year ending', aggfunc='count')#.reset_index().rename(columns={'Grantee': 'total_to_remove'})
t.columns = t.columns.get_level_values(1)

In [ ]:
t.to_csv('output/new_grantee_geo_distrib.csv', index=False)

In [ ]:
t[[2017, 2018, 2019, 2020]].sort_values(by=[2017],ascending=False).head(5)

### What percent of 2020 grantees had received a grant in a previous year?

In [ ]:
#g_not_20 = df.loc[df['Fiscal year ending'] != 2020]
g_not_20 = grants_df[(grants_df['Fiscal year ending'] != 2020) & (grants_df['Fiscal year ending'] != 2021)]
#create df with unique grantees and years in which they had a grant, with each year's total $ awarded
g_not_20_pv = g_not_20.pivot_table(index='Grantee', 
                      columns='Fiscal year ending', 
                      values='USD over grant life', 
                      aggfunc=np.sum).reset_index().rename_axis(None, axis=1)

#create df with unique grantees and total amount received
g_not_20_pv_simple = pd.pivot_table(data=g_not_20, index='Grantee', values='USD over grant life', aggfunc='sum').reset_index().rename(columns={'USD over grant life': 'total_usd'})

#get list of 2020 unique grantees
g20 = grants_df.loc[grants_df['Fiscal year ending'] == 2020]
g20list = list(g20['Grantee'].unique())

In [ ]:
# 2020 - What percent of 2020 grantees had received a grant in a previous year?
#identify which 2020 grantees had received a grant in a previous year

grantees_2020_that_received_prior_grant  = g_not_20_pv_simple[g_not_20_pv_simple['Grantee'].isin(g20list)]

print('grantees_2020_that_received_prior_grant:', len(grantees_2020_that_received_prior_grant)) 
print('2020 Grantees:', len(g20list))
print('2020 Grantees that received prior grants:', (len(grantees_2020_that_received_prior_grant)/len(g20list)*100), '%')

## What is the percent of new grants over the last years?

What percent of grants awarded to returning grantees?
What is the percent of new grantee grants over the last three years?

In [ ]:
total_grants = df.groupby('Fiscal year ending').size().to_frame().reset_index().rename(columns={0: 'total_granted'})
unique_grantees = df.groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'unique_grantees'})
unique_new_grantees = df[df['grant_count'] == 1].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'unique_n_grantees'})

#get a count of all grants awarded to a new grantee that has not received a grant in a previous year, not including those grantees that doubled up in their first
#year and received a follow-up grant
new_grantee_grants = pd.pivot_table(data=df[df['grant_count'] == 1], index='Fiscal year ending', values='counter', aggfunc='count').reset_index().rename(columns={'counter': 'total_n_grantee_grants'})

#returning grantee       = a grantee that has previously, in any previous year, received a WMF grant (not including returns in the same initial new grant year)
#returning grantee grant = a grant given to a grantee that has previously, in any previous year, received a WMF grant (not including returns in the same initial new grant year)
#returning grants        = a grant given to a grantee that has previously, at any time, received a grant

#new grantees that returned in the same year that should be removed from the count of returning grantees for that year
#...because they were counted in the new grantees for that year
#remove = df[((df[['Fiscal year ending', 'Grantee']].duplicated(keep=False)) & (df.loc[df['grant_count'] == 1].any(axis=1)))] #works only on python 3.5
dupes_mask = df.duplicated(subset=['Fiscal year ending','Grantee'], keep=False)
dupes = df[dupes_mask]
remove = dupes[dupes['grant_count'] == 1]

#get the number of returning grantees in year x minus the number of grantees that are returning in year x AND were also new that year 
#example: first grant in Jan and returned for a grant in June and October
#because this grantee was counted in the number of new grantees, we don't count them as returning grantees for this year
unique_returning_grantees = ((df[df['grant_count'] >= 2].groupby('Fiscal year ending')['Grantee'].nunique()) - (remove.groupby('Fiscal year ending')['Grantee'].nunique())).to_frame().reset_index().rename(columns={'Grantee': 'unique_r_grantees'})


#create roll_up df combining the dfs above
year_roll_up = total_grants.merge(unique_grantees, on='Fiscal year ending', how='left').merge(unique_new_grantees, on='Fiscal year ending', how='left').merge(unique_returning_grantees, on='Fiscal year ending', how='left')
year_roll_up['n_grantee %'] = year_roll_up['unique_n_grantees'] / year_roll_up['unique_grantees']

#get the number of grants given to returning grantees in year x minus the number of grants given to grantees that are returning in year x AND were also new that year 
#example: first grant in Jan and returned for a grant in June and October
#because this grantee was counted in the number of new grantees, we don't count them as returning grantees for this year

#get a count of all grants awarded to a grantee that has received more than 2 grants
returning_grants = pd.pivot_table(data=df[df['grant_count'] >= 2], index='Fiscal year ending', values='counter', aggfunc='count').reset_index().rename(columns={'counter': 'total_returning_grants'})
#get a count of all grants where the grantee and year are the same but one of those rows has a 1
returning_grantee_grants_prep2 = pd.pivot_table(data=remove, index='Fiscal year ending', values='Grantee', aggfunc='count').reset_index().rename(columns={'Grantee': 'total_to_remove'})
returning_grantee_grants_prep3 = returning_grants.merge(returning_grantee_grants_prep2, on='Fiscal year ending', how='left')

returning_grantee_grants_prep3['total_r_grantee_grants'] = returning_grantee_grants_prep3['total_returning_grants']-returning_grantee_grants_prep3['total_to_remove']
returning_grantee_grants = returning_grantee_grants_prep3[['Fiscal year ending','total_r_grantee_grants']]

#grantees
year_roll_up['r_grantee %'] = year_roll_up['unique_r_grantees'] / year_roll_up['unique_grantees']
#grants
year_roll_up = year_roll_up.merge(returning_grants, on='Fiscal year ending', how='left').merge(returning_grantee_grants, on='Fiscal year ending', how='left')
year_roll_up = year_roll_up.merge(new_grantee_grants, on='Fiscal year ending', how='left')
year_roll_up['r_grantee_grants %'] = year_roll_up['total_r_grantee_grants'] / year_roll_up['total_granted']
year_roll_up['n_grantee_grants %'] = year_roll_up['total_n_grantee_grants'] / year_roll_up['total_granted']

year_roll_up = year_roll_up.round(2)
year_roll_up

In [ ]:
#check data
rgrantees_2017 = df[df['Fiscal year ending']==2017]
rgrantees_2018 = df[df['Fiscal year ending']==2018]
rgrantees_2019 = df[df['Fiscal year ending']==2019]

In [ ]:
year_roll_up.to_csv('outputs/year_roll_up_df.csv', index=False)

In [ ]:
#plot number of unique new grantees by year
year_roll_up_df.plot(kind = 'bar', 
                     width=.75,
                     x='Fiscal year ending', 
                     y='unique_n_grantees', 
                     color='Blue', 
                     legend=False
                    );

In [ ]:
#plot % of unique new grantees by year
year_roll_up_df.plot(kind = 'bar', 
                     width=.75,
                     x='Fiscal year ending', 
                     y='n_grantee %', 
                     color='Blue', 
                     legend=False
                    );

In [ ]:
#plot % of new grantees by year
year_roll_up_df.plot(kind = 'bar', 
                     width=.75,
                     x='Fiscal year ending', 
                     y='n_grantee_grants %', 
                     color='Blue', 
                     legend=False
                    );

In [ ]:
#plot % of new grantees by year for years 2017-2021
year_roll_up_df[year_roll_up_df['Fiscal year ending'] >= 2017].plot(kind = 'bar', 
                                                                    width=.78,
                                                                    x='Fiscal year ending', 
                                                                    y='n_grantee_grants %', 
                                                                    color='#2a9df4', 
                                                                    legend=False
                                                                   );

In [ ]:
returning_grantee_grants_max = pd.pivot_table(data=df[df['grant_count'] >= 2], index='Fiscal year ending', values='grant_count', aggfunc='max').reset_index().rename(columns={'grant_count': 'max_grants_received_to_date'})
returning_grantee_grants_max

## Number of Grants from outside established comms

In [ ]:
grants_df.groupby(['Fiscal year ending', 'Community type'])['counter'].nunique().to_frame().rename(columns={'Counter': 'unique_grants'})

In [ ]:
len(grants_df[grants_df['Fiscal year ending'] == 2020])

## Number of Grantees from Emerging Communities

In [ ]:
grants_df['Community type'].unique()

In [ ]:
grants_df.groupby(['Fiscal year ending', 'Community type'])['Grantee'].nunique().to_frame().rename(columns={'Grantee': 'unique_grantees'})

In [ ]:
emerging_by_year_all = grants_df[grants_df['Community type'] == 'Emerging'].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'emerging_grantees'})
outside_developed_all = grants_df[(grants_df['Community type'] != 'Developed')].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'emerging_grantees_outside_developed'})

emerging_year_roll_up_all = unique_grantees.merge(emerging_by_year_all, on='Fiscal year ending', how='left')

emerging_year_roll_up_all = emerging_year_roll_up_all.merge(outside_developed_all,on='Fiscal year ending', how='left')
emerging_year_roll_up_all['emerging_%'] = emerging_year_roll_up_all['emerging_grantees'] / emerging_year_roll_up_all['unique_grantees']

emerging_year_roll_up_all['outside_dev_%'] = emerging_year_roll_up_all['emerging_grantees_outside_developed'] / emerging_year_roll_up_all['unique_grantees']

emerging_year_roll_up_all

## Number of New Grantees from Emerging Communities

In [ ]:
grants_df[grants_df['grant_count'] == 1].groupby(['Fiscal year ending', 'Community type'])['Grantee'].nunique().to_frame().rename(columns={'Grantee': 'unique_n_grantees'})

In [ ]:
new = grants_df[grants_df['grant_count'] == 1]

emerging_by_year = grants_df[grants_df['grant_count'] == 1][grants_df[grants_df['grant_count'] == 1]['Community type'] == 'Emerging'].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'emerging_n_grantees'})
outside_developed_new = new[(new['Community type'] != 'Developed')].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'emerging_n_grantees_outside_developed'})
emerging_year_roll_up = unique_new_grantees.merge(emerging_by_year, on='Fiscal year ending', how='left')

emerging_year_roll_up = emerging_year_roll_up.merge(outside_developed_new,on='Fiscal year ending', how='left')
emerging_year_roll_up['emerging_%'] = emerging_year_roll_up['emerging_n_grantees'] / emerging_year_roll_up['unique_n_grantees']

emerging_year_roll_up['outside_dev_%'] = emerging_year_roll_up['emerging_n_grantees_outside_developed'] / emerging_year_roll_up['unique_n_grantees']

emerging_year_roll_up

## % of rapid grant grantees that recieved a rapid grant in the year prior

In [ ]:
rapid_20 = grants_df[(grants_df['Program']=='Rapid') & (grants_df['Fiscal year ending']==2020)]
rapid_19 = grants_df[(grants_df['Program']=='Rapid') & (grants_df['Fiscal year ending']==2019)]

In [ ]:
r19list = list(rapid_19['Grantee'].unique())
r_grantees_2020_that_received_prior_r_grant  = rapid_20[rapid_20['Grantee'].isin(r19list)]

print('r_grantees_2020_that_received_prior_r_grant:', len(r_grantees_2020_that_received_prior_r_grant)) 
print('2019 rapid Grantees:', len(r19list))
print('2020 rapid Grantees:', len(rapid_20['Grantee'].unique()))
print('2020 R grant Grantees that received prior rapid grants in prior year:',(len(r_grantees_2020_that_received_prior_r_grant)/len(rapid_20['Grantee'].unique())*100), '%')

### Average # of rapid grants received 

In [ ]:
#Average num of rapid grants received by grantees that receive rapid grants


In [ ]:
#r_grants_received_by_grantee 

rgrbg = grants_df[grants_df['Program'] == 'Rapid'].groupby('Grantee').size().to_frame().reset_index().rename(columns={0: 'rapid_grants_received'})

rgrbg.mean()

In [ ]:
bins = [0, 1, 2, 3, 5, 10, 15, 20]
s = pd.cut(rgrbg['rapid_grants_received'], bins=bins).value_counts()
s

## avg r_grants_received_by_grantee in 20'

In [ ]:


twenty = grants_df[grants_df['Fiscal year ending'] == 2020]
rgrbg_20 = twenty[twenty['Program'] == 'Rapid'].groupby('Grantee').size().to_frame().reset_index().rename(columns={0: 'rapid_grants_received'})

rgrbg_20.mean()

## Rapid Grants received by rapid grantees in 2020

In [ ]:
#number of rapid grant grantees in 2020
total_rgrbg_20 = len(rgrbg_20)
total_rgrbg_20

In [ ]:
rgrbg_20_receiving_multiple = len(rgrbg_20[rgrbg_20['rapid_grants_received'] >= 2])
rgrbg_20_receiving_multiple

In [ ]:
rgrbg_20_receiving_multiple/total_rgrbg_20

## % of grantees who progress through the pipeline [rapid>project>SAPG/APG]

In [ ]:
#2016 forward
pipeline_df = df[df['Fiscal year ending'] >=2016]
#do not include TPS, PEG, IEG, Partnership Grants
exclude_pipeline = ['TPS', 'PEG', 'IEG', 'Partnership Grants']

simple = exclude + ['']
pipeline_df = pipeline_df[~pipeline_df['Program'].isin(exclude_pipeline)]

#pivot df so that each grantee gets a single row
pipeline_df['idx'] = pipeline_df.groupby('Grantee').cumcount()+1
pipeline_pvdf = pipeline_df.pivot(index='Grantee',columns='idx')['Program']
pipeline_pvdf = pd.DataFrame(ndf.to_records())
pipeline_pvdf = pipeline_pvdf.apply(lambda x : x.name+"_"+ x)
pipeline_pvdf['1'] = pipeline_pvdf['1'].str.split('\_').str[-1].str.strip();

In [ ]:
len(pipeline_df[pipeline_df.eq('APG').any(1)])

In [ ]:
len(pipeline_df[pipeline_df.eq('SAPG').any(1)])

In [ ]:
#create a sankey diagram
#https://chart-studio.plotly.com/~alishobeiri/1591/plotly-sankey-diagrams/#/


#projected mockup using random values

#FYI: 3 grants of any kind (ex:conf, events, rapid, project) > Simple APG
#include conference grants
#2016 forward

#do not include TPS, PEG, IEG, Partnership Grants
#WMS wiki mania scholarships
source = [0,0,0,0,0,0,       # Application          # target 1-7
          1, 1,             # Rapid                 #target 8-9
          2, 2,             # Project               #target 10-11
          3,                # APG                   #target 12
          4, 4, 4,          # SAPG                  #target 13-15
          5,5,5,            #Conference             #target 16-18
          6,6,              #WMS                    #target 19-21
         #22,23,24,25,26,27,
         ]          
                            
                             
target = [1, 2, 3, 4, 5, 6,
          7, 8, 9, 10, 11, 12, 13, 14, 
          15, 16, 17,18,19,
          20,21,
          
]

value = [484500, 468350, 355300, 306850, 339150, 145350, 371450,  # target 1-7
         96900, 129200, 
         80750, 48450, 
         48450, 
         32300, 16150, 113050, 
         129200, 6460,700, 
         400,900,500                                              #target 19-21
        ]

 
            
label = ['Application', #target 1
         'Rapid', #target 2
         'Project', #target 3
         'APG', #target 4
         'SAPG',#target 5
         'Conference',#target 6
         'WMS',#target 7
         
         '2_Rapid_Project', #target 8
         '2_Rapid_SAPG',
         
         '2_Project_Rapid', #target 10
         '2_Project_WMS',#target 11
         
         '2_APG_wikicite', #target 12
         
         '2_SAPG_wikicite',#SAPG > Wikicite #target 13
         '2_SAPG_SAPG',#SAPG > SAPG #target 14
         '2_SAPG_Conference',#SAPG > Conf #questions about which dept should be providing the grant; which affiliates are getting $ from which source
         
         '2_Conf_rapid',#Conf > rapid #target 16
         '2_Conf_SAPG',#Conf > SAPG #target 17
         '2_Conf_Conference',#Conf > conf #target 18
         
         '2_WMS_rapid',#WMS > Rapid #target 19
         '2_WMS_project',#WMS > Project (kind of an edge-case?) #target 20
         'wz'
        ]

link=dict(source=source, target=target, value=value)
node=dict(label=label, pad=15, thickness=5)
data=go.Sankey(link=link, node=node)
#plot
fig=go.Figure(data)
fig.show()


In [ ]:
#combine column values to create source to target df base
z = pipeline_pvdf.copy()
z['1'] =z['1'] +' '+ z['2']
z['2'] =z['2'] +' '+ z['3']
z['3'] =z['3'] +' '+ z['4']
z['4'] =z['4'] +' '+ z['5']
z['5'] =z['5'] +' '+ z['6']
z['6'] =z['6'] +' '+ z['7']
z['7'] =z['7'] +' '+ z['8']
z['8'] =z['8'] +' '+ z['9']
z['9'] =z['9'] +' '+ z['10']
z['10'] =z['10'] +' '+ z['11']
z['11'] =z['11'] +' '+ z['12']
z['12'] =z['12'] +' '+ z['13']
z['13'] =z['13'] +' '+ z['14']

#unstack
del z['Grantee']
z = z.unstack().to_frame().reset_index()
#select columns, split column & rename
z = z[[0]]
z[['source','dest']] = z[0].str.split(expand=True)
z = z[['source','dest']]
#drop rows with all NaNs
z = z.dropna(how='all')
#In order to create a sankey diagram we need the pairs and the corresponding counts.
counts=z.groupby(["source", "dest"]).size().reset_index(name='counts')



In [ ]:
past=counts['source'].tolist()

#Plotly's sankey diagram API needs a list of all labels, and links connecting the sources to the targets.
#Converting them to lists so that we can pass the indices to the API
current=counts['dest'].tolist()
unique_values=pd.unique(pd.concat((counts['source'],counts['dest']),axis=0)).tolist()

past_indices=[unique_values.index(i) for i in past]
current_indices=[unique_values.index(i) for i in current]
#type(past)

In [ ]:
#handle colors

#colors for Sankey diagram
#https://plotly.com/python/discrete-color/

colors = px.colors.qualitative.D3
node_colors_mappings = dict([(node,np.random.choice(colors)) for node in all_nodes])
node_colors = [node_colors_mappings[node] for node in all_nodes]
#edge_colors = [node_colors_mappings[node] for node in programs]

In [ ]:
data_trace = dict(
    type='sankey',
    #width = 1118,
    #height = 772,
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label=unique_values,
      #color =  "red",
      #color =  node_colors,
        color = node_color_values,
    ),
    link=dict(
        source=past_indices,
        target=current_indices,
        value=counts['counts'],
        #color = edge_colors,
    ))

layout =  dict(
    height=1500,
    #width=80,
    title = "Grantee Journey through Grants since 2016",
    font = dict(
      size = 10
    )
)


fig = dict(data=[data_trace], layout=layout)
ply.iplot(fig, validate=False)
